# Plotting the Shortest Path with [Dijkstra Algorithm](https://www.freecodecamp.org/news/dijkstras-shortest-path-algorithm-visual-introduction/)

In [ ]:
!pip install osmnx==0.16.1
!pip install geopandas==0.9.0
!pip install shapely==1.8.0
!pip install -U kaleido
!pip install networkx
!pip install plotly
!pip install pandas

In [ ]:
import os
import networkx as nx
import plotly.graph_objects as go
import osmnx as ox
import pandas as pd
import geopandas
from shapely.geometry import Point

def generate_path(origin_point, target_point, perimeter):
    origin_lat, origin_long = origin_point
    target_lat, target_long = target_point

    # Calculate boundaries
    north, south = max(origin_lat, target_lat), min(origin_lat, target_lat)
    east, west = max(origin_long, target_long), min(origin_long, target_long)

    # Construct road graph
    roadgraph = ox.graph_from_bbox(north + perimeter, south - perimeter,
                                   east + perimeter, west - perimeter,
                                   network_type='bike', simplify=False)

    # Get nearest nodes
    origin_node = ox.get_nearest_node(roadgraph, origin_point)
    target_node = ox.get_nearest_node(roadgraph, target_point)

    # Get optimal path via Dijkstra
    route = nx.shortest_path(roadgraph, origin_node, target_node,
                             weight='length', method='dijkstra')

    # Extract path coordinates
    lat = []
    long = []

    for i in route:
        point = roadgraph.nodes[i]
        long.append(point['x'])
        lat.append(point['y'])

    # Return the paths
    return long, lat

    return path_coordinates

In [ ]:
import folium

def plot_map(origin_point, target_points, long, lat):
    # Create a Folium map centered around the origin point
    map_center = [origin_point[0], origin_point[1]]
    m = folium.Map(location=map_center, zoom_start=12, control_scale=True)

    # Create FeatureGroups for different layers
    origin_layer = folium.FeatureGroup(name='Origin')
    destination_layer = folium.FeatureGroup(name='Destinations')
    path_layer = folium.FeatureGroup(name='Paths')

    # Add origin point marker with popup to the origin layer
    folium.Marker(location=origin_point, popup=f'Origin: {origin_point}', icon=folium.Icon(color='blue')).add_to(origin_layer)

    # Add target points markers with popups to the destination layer
    for i, target_point in enumerate(target_points):
        folium.Marker(location=target_point, popup=f'Destination {i+1}: {target_point}', icon=folium.Icon(color='green')).add_to(destination_layer)

    # Add optimal paths to the path layer
    for i in range(len(lat)):
        path = list(zip(lat[i], long[i]))
        folium.PolyLine(locations=path, color='red').add_to(path_layer)

    # Add FeatureGroups to the map
    origin_layer.add_to(m)
    destination_layer.add_to(m)
    path_layer.add_to(m)

    # Add Layer Control
    folium.LayerControl().add_to(m)

    # Save map
    m.save('dijkstra_map.html')

    # Display map
    display(m)

In [ ]:
# Data Import
df_coordinates = pd.read_csv('sample_data/coordinates.csv')
df = df_coordinates[['LATITUDE', 'LONGITUDE']]

# Create GeoDataFrame with point geometries
geometry = [Point(lon, lat) for lat, lon in zip(df['LATITUDE'], df['LONGITUDE'])]
geo_df = geopandas.GeoDataFrame(df, geometry=geometry)

# Format target geocoordinates
target_points = [(lat, lon) for lon, lat in zip(df['LONGITUDE'], df['LATITUDE'])]

# Set origin geocoordinate
origin_point = (49.78728, 9.97872)

# Calculate paths
paths_longitude = []
paths_latitude = []
perimeter = 0.10

for target_point in target_points:
    path_lon, path_lat = generate_path(origin_point, target_point, perimeter)
    paths_longitude.append(path_lon)
    paths_latitude.append(path_lat)

plot_map(origin_point, target_points, paths_longitude, paths_latitude)


## Haversine

Calculate the distance between the origin and all destination points using the Haversine formula, which calculates the great-circle distance between two points on the Earth's surface given their longitudes and latitudes.

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of the Earth in kilometers

    return distance

def calculate_distances(origin_point, target_points):
    distances = []
    for target_point in target_points:
        distance = haversine(origin_point[0], origin_point[1], target_point[0], target_point[1])
        distances.append(distance)
    return distances

In [ ]:
distances = calculate_distances(origin_point, target_points)
for i, distance in enumerate(distances):
    print(f"Distance from origin to destination {i+1}: {distance:.2f} km")